In [1]:
# import zipfile
# import os

# def extract_zip(zip_file, extract_to):
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)
#     print("Extraction complete.")

# # Replace 'your_zip_file.zip' with the name of your zip file
# zip_file = 'Dataset_Speech_Assignment.zip'

# # Replace 'your_extraction_directory' with the directory where you want to extract the files
# extract_to = r'/mnt/c/Users/Manish/Desktop/SpeechAssign3/SSL_Anti-spoofing/Dataset1'

# # Create the extraction directory if it doesn't exist
# os.makedirs(extract_to, exist_ok=True)

# # Extract the zip file
# extract_zip(zip_file, extract_to)


In [2]:
# ! pip install torch torchvision numpy scipy matplotlib scikit-learn pandas librosa

In [3]:
import os
import torch
import librosa
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

/home/manish/miniconda3/envs/speech/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ! pip install protobuf==3.20 tensorboard

In [5]:
import torch.nn as nn

In [6]:
#GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
print('Device: {}'.format(device))

# Load the SSL W2V model trained for LA and DF tracks
from model import Model

model = Model(None, device=device)
model = nn.DataParallel(model).to(device)
model.load_state_dict(torch.load('/mnt/c/Users/Manish/Desktop/SpeechAssign/SSL_Anti-spoofing/Best_LA_model_for_DF.pth'))

model.eval()

Device: cuda


DataParallel(
  (module): Model(
    (ssl_model): SSLModel(
      (model): Wav2Vec2Model(
        (feature_extractor): ConvFeatureExtractionModel(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU()
            )
            (1): Sequential(
              (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU()
            )
            (2): Sequential(
              (0):

In [7]:
# Define a function to preprocess the audio samples
def preprocess_audio(audio_path):
    # Load the audio file and extract features
    audio, sr = librosa.load(audio_path, sr=None)
    # Here you can apply further preprocessing if needed, e.g., feature extraction
    return audio

# Define paths to real and fake audio samples
real_audio_dir = "/mnt/c/Users/Manish/Desktop/SpeechAssign3/SSL_Anti-spoofing/Dataset1/Dataset_Speech_Assignment/Real"
fake_audio_dir = "/mnt/c/Users/Manish/Desktop/SpeechAssign3/SSL_Anti-spoofing/Dataset1/Dataset_Speech_Assignment/Fake"

# Collect paths to real and fake audio files
real_audio_paths = [os.path.join(real_audio_dir, filename) for filename in os.listdir(real_audio_dir)]
fake_audio_paths = [os.path.join(fake_audio_dir, filename) for filename in os.listdir(fake_audio_dir)]

# Classify the audio samples and collect predictions
predictions = []
for audio_path in real_audio_paths + fake_audio_paths:
    processed_audio = preprocess_audio(audio_path)
    processed_audio_tensor = torch.tensor(processed_audio, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        output = model(processed_audio_tensor)
    # Assuming the model output is a probability between 0 and 1
    prediction = output.item()  # Convert tensor to scalar
    predictions.append(prediction)

# Create ground truth labels (0 for real, 1 for fake)
ground_truth = np.zeros(len(real_audio_paths)) + np.ones(len(fake_audio_paths))

# Calculate AUC
auc_score = roc_auc_score(ground_truth, predictions)

# Calculate ROC curve
fpr, tpr, _ = roc_curve(ground_truth, predictions)

# Calculate EER
eer = 1.0
for i in range(len(fpr)):
    if fpr[i] == tpr[i]:
        eer = fpr[i]
        break

# Analyze the performance of the model
print("AUC:", auc_score)
print("EER:", eer)


/home/manish/miniconda3/envs/speech/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


NoBackendError: 